In [1]:
%matplotlib inline
%load_ext autoreload

import warnings
warnings.filterwarnings("ignore") # disable warnings

from os import getcwd
from os.path import join, abspath, pardir, exists
import numpy as np
import pandas as pd

import pickle, json

import matplotlib.pyplot as plt
import seaborn as sns

# plotly
import plotly.express as px
import plotly.figure_factory as ff
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# scipy
from scipy.stats import ttest_ind, chi2_contingency, boxcox, skew
from scipy.stats.stats import pearsonr

# sklearn libraries
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.impute import KNNImputer, SimpleImputer, MissingIndicator
from sklearn.cluster import DBSCAN
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import make_column_selector, make_column_transformer, make_column_transformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier


from sklearn.experimental import enable_iterative_imputer # enable experimental imputer
from sklearn.impute import IterativeImputer               # sample imputation
from sklearn import preprocessing                         # encoders, transformations
from sklearn.model_selection import cross_validate        # cross-validation, model evaluation
from sklearn.model_selection import GridSearchCV          # hyper-parameter tuning
from sklearn.linear_model import LogisticRegression       # logistic regression model
from sklearn.svm import SVC                               # support vector machine model
from sklearn.neighbors import KNeighborsClassifier        # k-nearest neighbours model
from sklearn.ensemble import GradientBoostingClassifier   # gradient boosting model
from sklearn.ensemble import VotingClassifier             # voting ensemble model
from sklearn.ensemble import StackingClassifier           # stacking ensemble model


# statsmodel
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# IPython
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

##### Config settings

In [2]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
model_dir = join(parent_dir, "models")
data_file = join(data_dir, "test.csv")

# For IPython

InteractiveShell.ast_node_interactivity = "all" # To show all output after each cell execution (instead of the last output)

# For pandas

pd.options.display.max_columns = 200 # display upto 200 columns (instead of default 20)
pd.options.display.max_rows = 200 # display upto 200 rows (instead of default 60)

#### Helper functions

In [3]:
def save_model(model, file_path: str) -> None:
    """
    Save model as a pickle file
    """
    with open(file_path, "wb") as file:
        pickle.dump(model, file)

def load_model(file_path: str):
    """
    Load model from a pickle file
    """
    with open(file_path, "rb") as file:
        return pickle.load(file)

def dataframe_to_csv(df: pd.DataFrame, file_path: str) -> None:
    """
    Save dataframe as .csv file
    """
    df.to_csv(file_path, index=False)

def plot_distribution(data, bins, title, xlabel, ylabel):
    """
    Plot distribution functions
    """
    ax = sns.distplot(
        data,
        bins=bins,
        hist_kws={
            "linewidth": 1,
            'edgecolor': 'black',
            'alpha': 1.0
            },
        kde=False
    )
    _ = ax.set_title(title)
    _ = ax.set_xlabel(xlabel)
    _ = ax.set_ylabel(ylabel)

def plot_relationship(x, y, title, xlabel, ylabel):
    """
    Plot relationship between two features
    """
    ax = sns.barplot(
        x=x,
        y=y,
        orient='h'
    )
    _ = ax.set_title(title)
    _ = ax.set_xlabel(xlabel)
    _ = ax.set_ylabel(ylabel)

def print_moments(title, feature):
    """
    Print a feature's mean, standard deviation, skewness and kurtosis
    """
    print(title)
    print('Mean: '+'{:>18.2f}'.format(feature.mean()))
    print('Standard deviation: '+'{:.2f}'.format(feature.std()))
    print('Skewness: '+'{:>14.2f}'.format(feature.skew()))
    print('Kurtosis: '+'{:>14.2f}'.format(feature.kurtosis()))

#### Load test data

In [4]:
df = pd.read_csv(data_file, encoding= 'ISO-8859-1')
df.head()

,iid,gender,wave,position,order,pid,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field_cd,race,imprace,imprelig,goal,date,go_out,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,satis_2,length,numdat_2
0,50,True,2,12,13,25.0,25.0,2.0,20.00,20.00,15.00,15.00,15.00,15.00,False,9.0,8.0,9.0,8.0,10.0,7.0,8.0,7.0,True,27.0,1.0,1.0,4.0,4.0,6.0,6.0,2.0,1.0,10.0,10.0,3.0,8.0,5.0,3.0,4.0,3.0,10.0,5.0,6.0,8.0,10.0,7.0,10.0,6.0,2.0,5.0,7.0,35.00,10.00,20.00,15.00,10.00,10.00,8.0,9.0,9.0,10.0,8.0,True,7.0,NaN,NaN,NaN,NaN,NaN,8.0,7.0,2.0,5.0,6.0,1.0,2.0
1,53,True,2,19,7,33.0,30.0,2.0,30.00,20.00,30.00,10.00,0.00,10.00,False,8.0,7.0,5.0,6.0,7.0,4.0,6.0,3.0,True,28.0,4.0,2.0,2.0,2.0,2.0,6.0,2.0,4.0,8.0,4.0,3.0,10.0,9.0,8.0,8.0,4.0,4.0,9.0,5.0,7.0,9.0,9.0,9.0,7.0,3.0,6.0,9.0,40.00,15.00,20.00,10.00,5.00,10.00,8.0,9.0,8.0,8.0,8.0,False,7.0,9.0,9.0,7.0,9.0,7.0,7.0,8.0,2.0,8.0,8.0,3.0,3.0
2,361,True,14,12,9,352.0,29.0,6.0,20.00,20.00,20.00,10.00,10.00,20.00,False,6.0,6.0,7.0,6.0,6.0,5.0,6.0,6.0,True,34.0,1.0,2.0,5.0,5.0,2.0,6.0,6.0,1.0,3.0,1.0,3.0,5.0,9.0,10.0,3.0,3.0,4.0,10.0,6.0,8.0,9.0,7.0,7.0,3.0,2.0,5.0,NaN,30.00,20.00,30.00,5.00,5.00,10.00,7.0,7.0,5.0,7.0,9.0,False,6.0,10.0,6.0,6.0,6.0,4.0,5.0,2.0,0.0,4.0,7.0,3.0,3.0
3,153,False,7,11,12,173.0,22.0,2.0,18.37,18.37,20.41,20.41,16.33,6.12,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,True,28.0,7.0,4.0,1.0,1.0,2.0,6.0,3.0,2.0,8.0,5.0,7.0,7.0,7.0,6.0,6.0,5.0,7.0,10.0,6.0,7.0,9.0,7.0,7.0,8.0,5.0,6.0,NaN,9.76,19.51,19.51,14.63,19.51,17.07,7.0,7.0,6.0,7.0,6.0,False,5.0,5.0,6.0,6.0,NaN,NaN,5.0,5.0,0.0,4.0,5.0,1.0,3.0
4,35,False,2,17,16,46.0,26.0,2.0,20.00,35.00,20.00,10.00,10.00,5.00,True,8.0,8.0,8.0,6.0,9.0,8.0,8.0,5.0,True,25.0,2.0,2.0,9.0,7.0,1.0,6.0,3.0,7.0,9.0,7.0,9.0,8.0,8.0,8.0,5.0,6.0,9.0,7.0,9.0,8.0,8.0,7.0,7.0,8.0,3.0,7.0,8.0,20.00,23.00,23.00,22.00,7.00,5.00,8.0,9.0,9.0,9.0,7.0,True,8.0,8.0,9.0,9.0,8.0,10.0,8.0,8.0,2.0,10.0,6.0,1.0,2.0


## Test Analysis

#### Load models

In [9]:
clf_gb = load_model(join(model_dir, "clf_gb.pkl"))
clf_knn = load_model(join(model_dir, "clf_knn.pkl"))
clf_logistic_regression = load_model(join(model_dir, "clf_logistic_regression.pkl"))
clf_stacking = load_model(join(model_dir, "clf_stacking.pkl"))
clf_svc = load_model(join(model_dir, "clf_svc.pkl"))
clf_voting = load_model(join(model_dir, "clf_voting.pkl"))
col_trans = load_model(join(model_dir, "col_trans.pkl"))

##### Basic checking

In [10]:
df['dec'].isnull().any()
((df.isna().sum()/len(df)) * 100).to_frame(name='missing %').sort_values(by=['missing %'], ascending=False).T

False

,expnum,match_es,shar_o,shar,numdat_2,length,satis_2,amb_o,amb,fun,fun_o,met,prob_o,prob,sinc_o,intel_o,intel,like_o,like,sinc,attr,attr_o,pf_o_sha,age_o,pf_o_amb,pf_o_fun,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,race_o,pf_o_att,pf_o_sin,pf_o_int,career_c,exphappy,date,shar1_1,age,amb1_1,fun1_1,intel1_1,sinc1_1,attr1_1,shopping,yoga,art,music,museums,imprace,imprelig,goal,go_out,concerts,tvsports,exercise,dining,sports,gaming,clubbing,reading,tv,theater,movies,hiking,field_cd,race,pid,wave,position,order,gender,dec_o,met_o,dec,iid
missing %,77.724741,14.638027,12.967383,12.728719,11.296738,11.137629,11.137629,8.512331,8.035004,4.375497,4.295943,3.89817,3.579952,3.18218,3.18218,2.943516,2.863962,2.704853,2.625298,2.545744,1.988862,1.909308,1.909308,1.511535,1.431981,1.352426,1.272872,1.272872,1.272872,1.272872,1.272872,1.272872,1.272872,1.272872,1.272872,1.034208,0.875099,0.795545,0.795545,0.71599,0.71599,0.71599,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436,0.556881,0.477327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Imputate missing values

In [17]:
scaled = col_trans.transform(df)
test_df = pd.DataFrame(scaled, columns=df.columns)
test_df

,iid,gender,wave,position,order,pid,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field_cd,race,imprace,imprelig,goal,date,go_out,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,satis_2,length,numdat_2
0,50.0,1.0,2.0,12.0,13.0,25.0,25.0,2.0,20.00,20.00,15.00,15.00,15.00,15.00,0.0,9.0,8.0,9.0,8.0,10.0,7.0,8.0,7.0,1.0,27.0,1.0,1.0,4.0,4.0,6.0,6.0,2.0,1.0,10.0,10.0,3.0,8.0,5.0,3.0,4.0,3.0,10.0,5.0,6.0,8.0,10.0,7.0,10.0,6.0,2.0,5.0,7.000000,35.00,10.00,20.00,15.00,10.00,10.00,8.0,9.0,9.0,10.0,8.0,1.0,7.0,7.312198,9.213262,9.351184,9.287481,6.340726,8.0,7.0,2.0,5.0,6.0,1.0,2.0
1,53.0,1.0,2.0,19.0,7.0,33.0,30.0,2.0,30.00,20.00,30.00,10.00,0.00,10.00,0.0,8.0,7.0,5.0,6.0,7.0,4.0,6.0,3.0,1.0,28.0,4.0,2.0,2.0,2.0,2.0,6.0,2.0,4.0,8.0,4.0,3.0,10.0,9.0,8.0,8.0,4.0,4.0,9.0,5.0,7.0,9.0,9.0,9.0,7.0,3.0,6.0,9.000000,40.00,15.00,20.00,10.00,5.00,10.00,8.0,9.0,8.0,8.0,8.0,0.0,7.0,9.000000,9.000000,7.000000,9.000000,7.000000,7.0,8.0,2.0,8.0,8.0,3.0,3.0
2,361.0,1.0,14.0,12.0,9.0,352.0,29.0,6.0,20.00,20.00,20.00,10.00,10.00,20.00,0.0,6.0,6.0,7.0,6.0,6.0,5.0,6.0,6.0,1.0,34.0,1.0,2.0,5.0,5.0,2.0,6.0,6.0,1.0,3.0,1.0,3.0,5.0,9.0,10.0,3.0,3.0,4.0,10.0,6.0,8.0,9.0,7.0,7.0,3.0,2.0,5.0,0.019934,30.00,20.00,30.00,5.00,5.00,10.00,7.0,7.0,5.0,7.0,9.0,0.0,6.0,10.000000,6.000000,6.000000,6.000000,4.000000,5.0,2.0,0.0,4.0,7.0,3.0,3.0
3,153.0,0.0,7.0,11.0,12.0,173.0,22.0,2.0,18.37,18.37,20.41,20.41,16.33,6.12,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,1.0,28.0,7.0,4.0,1.0,1.0,2.0,6.0,3.0,2.0,8.0,5.0,7.0,7.0,7.0,6.0,6.0,5.0,7.0,10.0,6.0,7.0,9.0,7.0,7.0,8.0,5.0,6.0,0.663979,9.76,19.51,19.51,14.63,19.51,17.07,7.0,7.0,6.0,7.0,6.0,0.0,5.0,5.000000,6.000000,6.000000,4.565951,5.710605,5.0,5.0,0.0,4.0,5.0,1.0,3.0
4,35.0,0.0,2.0,17.0,16.0,46.0,26.0,2.0,20.00,35.00,20.00,10.00,10.00,5.00,1.0,8.0,8.0,8.0,6.0,9.0,8.0,8.0,5.0,1.0,25.0,2.0,2.0,9.0,7.0,1.0,6.0,3.0,7.0,9.0,7.0,9.0,8.0,8.0,8.0,5.0,6.0,9.0,7.0,9.0,8.0,8.0,7.0,7.0,8.0,3.0,7.0,8.000000,20.00,23.00,23.00,22.00,7.00,5.00,8.0,9.0,9.0,9.0,7.0,1.0,8.0,8.000000,9.000000,9.000000,8.000000,10.000000,8.0,8.0,2.0,10.0,6.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,473.0,0.0,19.0,9.0,7.0,492.0,30.0,6.0,30.00,0.00,30.00,30.00,10.00,0.00,0.0,8.0,10.0,10.0,2.0,8.0,10.0,8.0,1.0,1.0,34.0,10.0,4.0,1.0,1.0,2.0,6.0,4.0,2.0,7.0,5.0,5.0,8.0,8.0,8.0,8.0,6.0,6.0,9.0,8.0,8.0,9.0,9.0,9.0,7.0,7.0,7.0,8.879279,15.00,20.00,20.00,15.00,15.00,15.00,7.0,9.0,8.0,10.0,7.0,1.0,8.0,10.000000,10.000000,8.000000,9.000000,9.000000,9.0,7.0,0.0,3.0,9.0,3.0,3.0
1253,21.0,0.0,2.0,5.0,5.0,46.0,26.0,2.0,20.00,35.00,20.00,10.00,10.00,5.00,0.0,6.0,7.0,10.0,8.0,9.0,8.0,8.0,6.0,1.0,21.0,5.0,4.0,8.0,10.0,6.0,7.0,4.0,10.0,10.0,8.0,10.0,5.0,5.0,5.0,2.0,1.0,3.0,7.0,10.0,4.0,7.0,4.0,8.0,5.0,2.0,5.0,0.000000,12.00,18.00,20.00,16.00,16.00,18.00,5.0,9.0,7.0,9.0,10.0,0.0,6.0,8.000000,8.000000,6.000000,8.000000,5.000000,8.0,2.0,2.0,0.0,6.0,3.0,3.0
1254,525.0,0.0,21.0,17.0,11.0,549.0,28.0,2.0,20.00,20.00,20.00,20.00,0.00,20.00,0.0,6.0,8.0,8.0,9.0,7.0,7.0,5.0,5.0,1.0,24.0,15.0,2.0,2.0,8.0,2.0,4.0,1.0,3.0,6.0,2.0,8.0,6.0,5.0,5.0,3.0,2.0,1.0,8.0,8.0,9.0,10.0,7.0,8.0,8.0,1.0,6.0,4.315891,10.00,15.00,30.00,20.00,15.00,10.00,6.0,10.0,8.0,10.0,9.0,0.0,8.0,8.000000,6.000000,7.000000,7.000000,6.000000,6.0,5.0,0.0,3.0,6.0,1.0,2.0
1255,529.0,0.0,21.0,3.0,1.0,536.0,29.0,2.0,30.00,20.00,20.00,20.00,10.00,0.00,0.0,6.0,9.0,9.0,6.0,8.0,7.0,3.0,6.0,1.0,22.0,5.0,4.0,4.0,1.0,2.0,4.0,1.0,5.0,7.0,4

In [18]:
test_df.isnull().any() # check missing values

iid         False
gender      False
wave        False
position    False
order       False
pid         False
age_o       False
race_o      False
pf_o_att    False
pf_o_sin    False
pf_o_int    False
pf_o_fun    False
pf_o_amb    False
pf_o_sha    False
dec_o       False
attr_o      False
sinc_o      False
intel_o     False
fun_o       False
amb_o       False
shar_o      False
like_o      False
prob_o      False
met_o       False
age         False
field_cd    False
race        False
imprace     False
imprelig    False
goal        False
date        False
go_out      False
career_c    False
sports      False
tvsports    False
exercise    False
dining      False
museums     False
art         False
hiking      False
gaming      False
clubbing    False
reading     False
tv          False
theater     False
movies      False
concerts    False
music       False
shopping    False
yoga        False
exphappy    False
expnum      False
attr1_1     False
sinc1_1     False
intel1_1    False
fun1_1    

Imputation model transformed `test` data successfully

In [19]:
def preprocessing_pipeline(data):
    """
    Pre-processing Pipeline for testing data
    """
    # Encode nominal features using one-hot encoding
    features_nominal = data.dtypes[data.dtypes == 'category'].index.values
    data = pd.get_dummies(data, prefix=features_nominal)

    # Calculate the average attribute ratings for each subject
    subject_attractiveness_mean = data[['iid', 'attr_o']].groupby(['iid']).mean()['attr_o']
    subject_sincerity_mean = data[['iid', 'sinc_o']].groupby(['iid']).mean()['sinc_o']
    subject_intelligence_mean = data[['iid', 'intel_o']].groupby(['iid']).mean()['intel_o']
    subject_fun_mean = data[['iid', 'fun_o']].groupby(['iid']).mean()['fun_o']
    subject_ambition_mean = data[['iid', 'amb_o']].groupby(['iid']).mean()['amb_o']
    subject_shared_interest_mean = data[['iid', 'shar_o']].groupby(['iid']).mean()['shar_o']

    # Insert average attribute ratings into dataframe
    data = data.merge(
        right=subject_attractiveness_mean,
        how='inner',
        on='iid'
    ).rename(columns={
        'attr_o_x': 'attr_o',
        'attr_o_y': 'subject_attractiveness_mean'
    })
    data = data.merge(
        right=subject_sincerity_mean,
        how='inner',
        on='iid'
    ).rename(columns={
        'sinc_o_x': 'sinc_o',
        'sinc_o_y': 'subject_sincerity_mean'
    })
    data = data.merge(
        right=subject_intelligence_mean,
        how='inner',
        on='iid'
    ).rename(columns={
        'intel_o_x': 'intel_o',
        'intel_o_y': 'subject_intelligence_mean'
    })
    data = data.merge(
        right=subject_fun_mean,
        how='inner',
        on='iid'
    ).rename(columns={
        'fun_o_x': 'fun_o',
        'fun_o_y': 'subject_fun_mean'
    })
    data = data.merge(
        right=subject_ambition_mean,
        how='inner',
        on='iid'
    ).rename(columns={
        'amb_o_x': 'amb_o',
        'amb_o_y': 'subject_ambition_mean'
    })
    data = data.merge(
        right=subject_shared_interest_mean,
        how='inner',
        on='iid'
    ).rename(columns={
        'shar_o_x': 'shar_o',
        'shar_o_y': 'subject_shared_interest_mean'
    })

    # Calculate difference between subject and partner's ages
    data['age_difference'] = abs(data['age'] - data['age_o'])

    #Calculate difference between subject's attribute ratings and partner's attributes ratings
    data['attractiveness_difference'] = abs(data['attr'] - data['attr_o'])
    data['sincerity_difference'] = abs(data['sinc'] - data['sinc_o'])
    data['intelligence_difference'] = abs(data['intel'] - data['intel_o'])
    data['fun_difference'] = abs(data['fun'] - data['fun_o'])
    data['ambition_difference'] = abs(data['amb'] - data['amb_o'])
    data['shared_interest_difference'] = abs(data['shar'] - data['shar_o'])

    #Scale normal features to zero mean and unit variance
    features_normal = [
        'attr_o',
        'sinc_o',
        'intel_o',
        'fun_o',
        'amb_o',
        'shar_o',
        'age_difference',
        'attractiveness_difference',
        'sincerity_difference',
        'intelligence_difference',
        'fun_difference',
        'ambition_difference',
        'shared_interest_difference'
    ]

    data[features_normal] = data[features_normal].apply(lambda x: preprocessing.scale(x))

    # Drop some features
    # Drop irrelevant features which contain no information about the target variable
    features_no_information = [
        'iid',
        'pid',
        'wave',
        'position',
        'order'
    ]
    # Drop features that are known in the future
    features_future_information = [
        'dec',
        #'dec_o',
        'like',
        'prob',
        'like_o',
        'prob_o'
    ]

    # Drop features that have low variance
    feature_variances = data.std().sort_values(ascending=True)
    features_low_variance = feature_variances[feature_variances < 0.1].index.values.tolist()

    # Drop features that have weak correlation with target variable
    correlations = data.corr().abs().unstack().sort_values(ascending=False).drop_duplicates()
    correlations = correlations[correlations != 1]
    partner_decision_correlations = correlations.loc['dec_o']
    features_weak_correlation = partner_decision_correlations[partner_decision_correlations < 0.1].axes[0].to_list()
    features_weak_correlation = list(set(features_weak_correlation) - set(features_future_information) - set(features_no_information))

    # Drop features that were used in interaction variables
    features_interaction = [
        'age',
        'age_o',
    ]

    features_remove = features_no_information + features_future_information + features_low_variance + features_weak_correlation + features_interaction
    data.drop(columns=features_remove, inplace=True)

    return data

In [20]:
test_df = preprocessing_pipeline(test_df)
test_df.head()

In [22]:
X_test, y_test = test_df.loc[:,~test_df.columns.isin(['dec_o'])], test_df['dec_o']

### 1. Testing on Baseline Models

#### 1.1. [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [23]:
y_pred = clf_logistic_regression.predict(X_test)
y_pred

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.